<a href="https://colab.research.google.com/github/hiro-hd/pytorch_advanced/blob/master/chap1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data


In [0]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [0]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [0]:
!ls /content/drive/"My Drive"/pytorch_advanced/1_image_classification/data

hymenoptera_data  imagenet_class_index.json


In [0]:
import sys
sys.path

sys.path.append("/content/drive/My Drive/pytorch_advanced/1_image_classification")

In [0]:
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# アリとハチの画像へのファイルパスのリストを作成する
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

/content/drive/My Drive/pytorch_advanced/1_image_classification/data/hymenoptera_data/train/**/*.jpg
/content/drive/My Drive/pytorch_advanced/1_image_classification/data/hymenoptera_data/val/**/*.jpg


In [0]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [0]:
#model

use_pretrained = True

net = models.vgg16(pretrained=use_pretrained)
net.train()

print("network defined, set to train mode")

network defined, set to train mode


In [0]:
#loss func

criterion = nn.CrossEntropyLoss()

In [0]:
#optimization/これはモデルに定義される。　optimizerに反映される

#パラメータを格納するリスト
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

#パラメータ名
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias",
                        "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
  if update_param_names_1[0] in name:
    param.requires_grad = True
    params_to_update_1.append(param)
    
    print("1: ", name)
    
  elif name in update_param_names_2:
    param.requires_grad = True
    params_to_update_2.append(param)
    
    print("2: ", name)
    
  elif name in update_param_names_3:
    param.requires_grad = True
    params_to_update_3.append(param)
    
    print("3: ", name)
    
  else:
    param.requires_grad = False
    
    print("N/A")

1:  features.0.weight
1:  features.0.bias
1:  features.2.weight
1:  features.2.bias
1:  features.5.weight
1:  features.5.bias
1:  features.7.weight
1:  features.7.bias
1:  features.10.weight
1:  features.10.bias
1:  features.12.weight
1:  features.12.bias
1:  features.14.weight
1:  features.14.bias
1:  features.17.weight
1:  features.17.bias
1:  features.19.weight
1:  features.19.bias
1:  features.21.weight
1:  features.21.bias
1:  features.24.weight
1:  features.24.bias
1:  features.26.weight
1:  features.26.bias
1:  features.28.weight
1:  features.28.bias
2:  classifier.0.weight
2:  classifier.0.bias
2:  classifier.3.weight
2:  classifier.3.bias
3:  classifier.6.weight
3:  classifier.6.bias


In [0]:
#optimizer
optimizer = optim.SGD([
    {'params':params_to_update_1, 'lr':1e-4},
    {'params':params_to_update_2, 'lr':5e-4},
    {'params':params_to_update_3, 'lr':1e-3}
], momentum=0.9)

In [0]:

#function to train the model

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
  
  #GPU
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")    #cuda:0 これはgpuデバイスの名前
  print("使用デバイス：　", device)
   
  net.to(device)  #load network to GPU
  
  torch.backends.cudnn.benchmark = True
  
  
  for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch, num_epochs))
    print("-"*10)
    
    for phase in ['train', 'val']:
      if phase=='train':
        net.train()
      else:
        net.eval()
      
      epoch_loss = 0.0
      epoch_corrects = 0
      
      if (epoch == 0) and (phase == 'train'):  #訓練は省略、何もプリントしないの？
        continue
      
      for inputs, labels in tqdm(dataloaders_dict[phase]):
        
        inputs = inputs.to(device)
        
        labels = torch.tensor(labels)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(phase == 'train'):  #torch.set_grad_enabledとは？
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          _, preds = torch.max(outputs, 1)  #tensor と long tensor?
          
          if phase == 'train':
            loss.backward()
            optimizer.step()
            
          epoch_loss += loss.item() * inputs.size(0)
          
          epoch_corrects += torch.sum(preds == labels.data)
          
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc = epoch_corrects.double(
      ) / len(dataloaders_dict[phase].dataset)
      
      print('{} Loss: {: 4f} Acc: {: 4f}'.format(
      phase, epoch_loss, epoch_acc))
     
     

In [0]:
num_epochs = 3
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

  0%|          | 0/5 [00:00<?, ?it/s]

使用デバイス：　 cuda:0
Epoch 0/3
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  0%|          | 0/8 [00:00<?, ?it/s]

val Loss:  3.613303 Acc:  0.431373
Epoch 1/3
----------


  0%|          | 0/5 [00:00<?, ?it/s]

...

val Loss:  0.953966 Acc:  0.712418
Epoch 2/3
----------


  0%|          | 0/5 [00:00<?, ?it/s]

train Loss:  0.603097 Acc:  0.798354


100%|██████████| 5/5 [00:02<00:00,  1.74it/s]

val Loss:  0.740274 Acc:  0.784314


In [0]:
save_path = '/content/drive/My Drive/pytorch_advanced/1_image_classification/weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)


In [5]:
!ls /content/drive/"My Drive"/pytorch_advanced/1_image_classification

1-1_load_vgg.ipynb	     make_folders_and_data_downloads.ipynb
1-3_transfer_learning.ipynb  utils
1-5_fine_tuning.ipynb	     weights_fine_tuning.pth
data


In [10]:
%cd /content/drive/"My Drive"/pytorch_advanced

/content/drive/My Drive/pytorch_advanced


In [9]:
!git init

Reinitialized existing Git repository in /content/.git/


In [0]:
git add 